<img src="../images/RC_Logo_EN_0128px.png">
# Bewertung eines Multi-Memory-Express
** ------   **
**14.8.2017**
Christian Menn, Daniel Oeltz


In [ ]:
import pyvacon.analytics as analytics
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import math
import copy
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
import pyvacon.marketdata.converter as mkt_converter
import pyvacon.instruments.converter as ins_converter
import pyvacon.pricing.tools as pricing_tools
import pyvacon.models.converter as model_converter
import qgrid
import ipywidgets as widgets
from IPython.display import Markdown, display
import pandas as pd
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
#%matplotlib notebook

refdate = analytics.ptime(2017,1,1,0,0,0) #dates which enters analytics objects must be analytics ptimes. 

## Übersicht
- Aufsetzen der Marktdaten
- Anlegen der Instrumentspezifikation
- Setzen der PricingData
- Bewertung (Local Volatility)
    - Berechnung des Preises
    - Exposure
    - Projektion bzgl. Korrelationen TODO
- Bewertung (Stochastic Volatility)
    - Berechnung des Preises

# Marktdaten
Die Marktdaten setzen sich zusammen aus
- Underlyingdaten
    - Spot
    - Borrow
    - Dividende
    - Volfläche
- Zinskurven
- Korrelationen

### Underlyingdaten

In [ ]:
underlying={
    'ADS':{
            'SPOT': 100,
            'DIVIDENDS': 
                  pd.DataFrame({ 
                      'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                      'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                      'CASH': [5.0, 5.0, 3.0, 2.0],
                      'YIELD': [0.0, 0.0, 0.02, 0.03],
                      'TAX': [0.8, 0.8, 0.8, 0.8]
                  }),
            'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
            'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              },
            
    'DAN':{
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2017,2,1), dt.datetime(2018,2,1), dt.datetime(2019,2,1), dt.datetime(2020,2,1)],
                  'PAYDATES': [dt.datetime(2017,2,1), dt.datetime(2018,2,1), dt.datetime(2019,2,1), dt.datetime(2020,2,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.0, 0.0],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
        },
    'DBK':{
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.02, 0.03],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              },
    'EON':{
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.02, 0.03],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              },
    'KS': {
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.02, 0.03],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              },
    'SAP':{
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.02, 0.03],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              },
    'SIE':{
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.02, 0.03],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              },
    'THY':{
            'SPOT': 100,
            'DIVIDENDS': 
              pd.DataFrame({ 
                  'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                  'CASH': [5.0, 5.0, 3.0, 2.0],
                  'YIELD': [0.0, 0.0, 0.02, 0.03],
                  'TAX': [0.8, 0.8, 0.8, 0.8]
              }),
              'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.01, 0.01]
              },
              'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.2, 0.25, 0.28, 0.3], 
                  'RHO':-0.65, 
                  'ETA':0.8, 
                  'GAMMA': 0.5
                  }
              }
}

discount = {
    'EUR':{
        'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
        'RATES' : [0.00, 0.00]
    }
}

In [ ]:
udl_selected = 'ADS'
q = qgrid.show_grid(underlying[udl_selected]['DIVIDENDS'])
w = widgets.Dropdown(options=list(underlying.keys()), value=udl_selected)
#help(qgrid)
#display(w,q)

def on_change(change):
    global udl_selected
    global w
    global q
    if change['type'] == 'change' and change['name'] == 'value':
        #underlying[udl_selected]['DIVIDENDS']  = spy.to_dict()
        udl_selected = change['new'] 
        #spy = pd.DataFrame(underlying[udl_selected]['DIVIDENDS'], columns =['EXDATES', 'PAYDATES', 'CASH', 'YIELD',  'TAX'])
        q.close()
        w.close()
        q=qgrid.show_grid(underlying[udl_selected]['DIVIDENDS'])
        q.open()
        w = widgets.Dropdown(options=list(underlying.keys()), value=udl_selected)
        w.observe(on_change)
        w.open()
        display(w,q)

w.observe(on_change)

In [ ]:
display(w,q)

### Korrelationen

In [ ]:
correlation_np = np.array(
    [ 
        [ 1.0, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8 ],
        [ 0.8, 1.0, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8 ],
        [ 0.8, 0.8, 1.0, 0.8, 0.8, 0.8, 0.8, 0.8 ],
        [ 0.8, 0.8, 0.8, 1.0, 0.8, 0.8, 0.8, 0.8 ],
        [ 0.8, 0.8, 0.8, 0.8, 1.0, 0.8, 0.8, 0.8 ],
        [ 0.8, 0.8, 0.8, 0.8, 0.8, 1.0, 0.8, 0.8 ],
        [ 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 1.0, 0.8 ],
        [ 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 1.0 ]
    ]
)
correlation = converter.from_np_matrix(correlation_np)
#uncomment the following to plot a heatmap of correlation matrix
#plt.matshow(correlation_np, cmap='hot')
#plt.colorbar()

### Volatilitäten: Sticky-Strike, Sticky-Moneyness, Sticky-X-Strike

Das Volflächenobjekt enthält die Referenz-Forwardkurve und 
Gegeben Expiry $T$, Strike $K$, diskontierte Cashdividenden $C_0$ und $C_1$ sowie und Forwards $F_0$ und $F_1$ zu Zeitpunkten $t_0<t_1$.
  - Sticky-Strike: 
  $\sigma_1(K) = \sigma_0(K)$
  - Sticky-Moneyness: $\sigma_1(K) = \sigma_0\left(\frac{F_0K}{F_1}\right)$
  - Sticky-X-Strike: $\sigma_1(K) = \sigma_0\left(\frac{(F_0-D_0)(K-D_1)}{(F_1-D_1)} + D_0\right)$
 
** Keine Cashdividenden: Sticky-Moneyness = Sticky-Strike **

Insbesondere relevant für Delta-Berechnung.

In [ ]:
spot_shift=1.2
cash_div = 2.0
yield_div = 0.03
udl = 'DAN'
def plot(x_axes):
    vol = mkt_converter.vol_from_dict(underlying, discount, udl, 'EUR', refdate)
    expiry = converter.getLTime(180, refdate)
    stickyness_types = [enums.VolatilityStickyness.StickyXStrike,
                        enums.VolatilityStickyness.StickyStrike, 
                        enums.VolatilityStickyness.StickyFwdMoneyness]
    xstrikes = pl.frange(0.5,1.5,0.01)
    underlying_shifted = copy.deepcopy(underlying)
    underlying_shifted[udl]['SPOT'] *= spot_shift
    underlying_shifted[udl]['DIVIDENDS'].loc[:,'CASH'] = cash_div
    underlying_shifted[udl]['DIVIDENDS'].loc[:,'YIELD'] = yield_div
       
    fwd_shifted = mkt_converter.fwd_from_dict(underlying_shifted[udl], udl, discount, 'EUR', refdate)
    fwd = mkt_converter.fwd_from_dict(underlying[udl], udl, discount, 'EUR', refdate)
    fwd_value = fwd_shifted.value(refdate, expiry)
          
    print('new forward: ' + str(fwd_value))
    print('old forward: ' + str(fwd.value(refdate, expiry)))
    shifted_vol = analytics.VolatilitySurface_createVolatilitySurfaceShiftedFwd(vol, fwd_shifted)

    for i, assumption in enumerate(stickyness_types):
        analytics.VolatilitySurface.setStickynessAssumption(assumption)
        
        tmp = []
        
        discounted_cash = fwd_shifted.discountedFutureCashDivs(refdate, expiry)
        x_axes_values = {
            'fwd moneyness' : [analytics.computeRealStrike(x, fwd_value, discounted_cash)/fwd_value for x in xstrikes],
            'strike' : [analytics.computeRealStrike(x, fwd_value, discounted_cash) for x in xstrikes],
            'x strike' : xstrikes
        }

        for x in xstrikes:
            tmp.append(shifted_vol.calcImpliedVol(refdate, expiry, x))
        plt.plot(x_axes_values[x_axes],tmp,'-x', label = assumption)
        plt.xlabel(x_axes)
        plt.ylabel('implied volatility')
        plt.legend()
        plt.title('spotshift:' + str(spot_shift) + ', cash div: ' + str(cash_div) + ', yield div: ' + str(yield_div))
        #if i < len(stickyness_types)-1:
         #   plt.figure()
plot('fwd moneyness') # 'strike', 'x strike', 'fwd moneyness'

# Instrument-Spezifikation
In der Library werden alle statischen, bewertungsrelevanten Daten in sogenannten Specifications spezifiziert. Diese Specifications werden dann an den Pricer übergeben.

In [ ]:
spec = {
    'ID': 'DE000DK0E8V2',
    'EXPIRY' : dt.datetime(2021,5,5,0,0,0),
    'CURRENCY' : 'EUR',
    'NOTIONAL' : 1.0,
    'TYPE' : 'WorstOf',
    'BASKET' : #pd.DataFrame(
        {
        'UNDERLYINGS': ['ADS', 'DAN', 'DBK', 'EON', 'KS', 'SAP', 'SIE', 'THY'],
        'WEIGHTS' : [ 1.0/73.74, 1.0/63.89, 1.0/28.865, 1.0/13.99, 1.0/31.38, 1.0/67.92, 1.0/97.11, 1.0/24.84]
        },
    #),
    'PAYOFF' :{
        'SPOT' : [0.0, 0.5, 1.0],
        'VALUE' : [0.0, 1000.0, 1000.0]
    },
    'COUPONS' : 
    #pd.DataFrame(
        {
        'OBSERVATION_DATES' :  [dt.datetime(2016,5,4,0,0,0), dt.datetime(2017,5,5,0,0,0), 
                dt.datetime(2018,5,4,0,0,0), dt.datetime(2019,5,6,0,0,0), 
                 dt.datetime(2020,5,5,0,0,0), dt.datetime(2021,5,5,0,0,0)],
        'PAYMENT_DATES' : [ dt.datetime(2016,5,12,0,0,0), dt.datetime(2017,5,12,0,0,0), 
                            dt.datetime(2018,5,14,0,0,0), dt.datetime(2019,5,13,0,0,0), 
                            dt.datetime(2020,5,12,0,0,0), dt.datetime(2021,5,12,0,0,0)
                          ],
        'COUPON': 75.5,
        'UP': True,
        'LEVELS' : [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
        'REDPEMPTION_LEVEL' : [0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        'REDEMPTION_PRICE': 1000,
        'REDEMPTION_UP': True
    }
    #)
}

### Basisinformation-Überblick

In [ ]:
display(Markdown('**Rückzahlungstermin **' + spec['EXPIRY'].strftime("%d.%m.%Y")))
display(Markdown('Nennwert ' + str(spec['NOTIONAL'])))
coupons = pd.DataFrame(data = spec['COUPONS'], columns = ['OBSERVATION_DATES', 'PAYMENT_DATES', 'COUPON', 'PAYMENT_DATES' ])
coupons.columns=['Beobachtungstag', 'Rückzahlungstermin', 'Zinsbetrag',  
                 'Zinszahlungstag']
print(coupons.to_string())
#for i, udl in enumerate(spec['BASKET']['UNDERLYINGS'].keys()):
    

# Bewertung (Local Volatility)
- In der Library werden alle Bewertungen über sogenannte Pricer duchgeführt
- Jeder Pricer bietet zwei Schnittstellen
    - eine spezielle calcPrice Methode: Erhält alle bewertungsrelevanten Objekte (PricingData)
    - eine allgemeine calcPrice-Schnittstelle: Erhält Container (Marktdaten, Parameter) und fült intern daraus PricingData
- PricingData kann serialisiert werden->komfortable Analysemöglichkeit bei Fehlern


## PricingData
Jeder Pricer definiert seine eigene Klasse, die alle Informationen zur Bewertung enthält. Diese Klasse kann bei Bedarf serialisiert, d.h. in eine Datei rausgeschrieben werden.
PricingData enthält immer
- Spezifikation des Instruments
- Bewertungsdatum
- Relevante Marktdaten
- Bewertungsparameter
- Was gerechnet werden soll (sog. PricingRequest)

In [ ]:
def set_pricing_data(pd, ins_dict):
    pd.valDate = refdate
    pd.spec = ins_converter.multi_memory_express_from_dict(ins_dict)
    pd.dsc = mkt_converter.discount_from_dict('EUR', discount['EUR'], refdate)
    pd.param = analytics.MonteCarloPricingParameter()
    pd.param.mcParam.numberOfSimulations = 10000
    vols = analytics.vectorConstVolatilities(len(spec['BASKET']['UNDERLYINGS']))
    for i, key in enumerate(spec['BASKET']['UNDERLYINGS']):
        vols[i] = mkt_converter.vol_from_dict(underlying, discount, key,'EUR', refdate)
    pd.vols = vols
    pd.setCorrelations(correlation)
    pd.pricingRequest = analytics.PricingRequest()

pricing_data = analytics.LocalVolMonteCarloPricingData()
set_pricing_data(pricing_data, spec)
# die folgende Zeile einkommentieren, um die Daten auf Platte zu schreiben
# pricing_data.save('pricing_spec.json', pricing_data)

## Bewertung
### Einfacher Preiscall

In [ ]:
tic = dt.datetime.now()
pr = analytics.price(pricing_data)
print('runtime: {}'.format(dt.datetime.now() - tic))
#plot the price
pr.getPrice()

### Delta

In [ ]:
pricing_data.pricingRequest.setDeltaGamma(True)
pr = analytics.price(pricing_data)
analytics.BasePricingData.save('DE000DK0E8V2' + '.json', pricing_data)
deltas = pr.getDeltas()

In [ ]:
for x in deltas.iterkeys():
    print(x + ': ' + str(deltas[x]))

### Exposure-Profile
We can analyze all cashflows from the MC simulation. We just have to specify the times, where we are interested in future cashflows, in the PricingRequest. The cashflows are then contained in the PricingResult object.

In [ ]:
cf_dates = converter.createPTimeList(refdate, range(1230,1260, 30))
pricing_data.pricingRequest.setCashflowTimes(cf_dates)
pr = analytics.price(pricing_data)
# the results now contain cashflows time slices and 
# we can get from them the future exposure (discounted to today)
future_cash =  pr.getCashflowSlices().getFutureExposure() # pr.getCashflowSlices().getFutureExposure()

Plot the histogram of all future expected cashflows.

In [ ]:
time = 0
n, bins, patches = plt.hist(future_cash[time], 100, normed=0, facecolor='blue', alpha=0.75)
print(future_cash[time])

Some statistics...

In [ ]:
mean_exposure = []
quantile_exposure = []
percentile = 50
for x in future_cash:
    mean_exposure.append(np.mean(x))
    quantile_exposure.append(np.percentile(x,percentile))
plt.plot(converter.create_datetime_list(cf_dates), mean_exposure,'-x', label='mean')
plt.plot(converter.create_datetime_list(cf_dates), quantile_exposure,'-^', label=str(percentile) + '% percentile')
plt.legend()

### Projektionen

In [ ]:
shifts = {
    'REDPEMPTION_LEVEL': [0.95, 0.96, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03],
    'LEVELS' : [0.95, 0.96, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03]
}

def plot_spec_proj(pr_data, ins_dict, shift):
    tmp = []
    for i in  range(len(shifts[shift])):
        new_spec = copy.deepcopy(ins_dict)
        coupons = new_spec['COUPONS'] 
        for x in coupons:
            for j in range(len(coupons['REDPEMPTION_LEVEL'])):
                coupons[shift][j] *= shifts[shift][i]
        pr_data.spec = ins_converter.multi_memory_express_from_dict(new_spec)
        pr = analytics.price(pr_data)
        tmp.append(pr.getPrice())
    plt.plot(shifts[shift], tmp,'-x')
    plt.xlabel(shift)
    plt.ylabel('price')
    

pricing_data.pricingRequest = analytics.PricingRequest()
plot_spec_proj(pricing_data, spec, 'REDPEMPTION_LEVEL')
plt.figure()
plot_spec_proj(pricing_data, spec, 'LEVELS')


# Bewertung (Stochastic Volatility)

## PricingData
Wie bei der Bewertung mittels Local Volatility sind die entsprechenden Bewertungsdaten aufzusetzen.

In [ ]:
stochvol_pricing_data = analytics.StochasticVolMonteCarloPricingData()
set_pricing_data(stochvol_pricing_data, spec)

### Aufsetzen Stochastic Volatility Modell

In [ ]:
heston = {
    'ADS':{
        'OBJ_ID': 'HESTON_ADS',
        'TYPE': 'HESTON',
        'S0': 1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'DAN':{
        'OBJ_ID': 'HESTON_DAN',
        'TYPE': 'HESTON',
        'S0':1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'DBK':{
        'OBJ_ID': 'HESTON_DBK',
        'TYPE': 'HESTON',
        'S0':1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'EON':{
        'OBJ_ID': 'HESTON_EON',
        'TYPE': 'HESTON',
        'S0' : 1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'KS':{
        'OBJ_ID': 'HESTON_KS',
        'TYPE': 'HESTON',
        'S0' : 1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'SAP':{
        'OBJ_ID': 'HESTON_SAP',
        'TYPE': 'HESTON',
        'S0' : 1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'SIE':{
        'OBJ_ID': 'HESTON_SIE',
        'TYPE': 'HESTON',
        'S0' : 1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    },
    'THY':{
        'OBJ_ID': 'HESTON_THY',
        'TYPE': 'HESTON',
        'S0' : 1.0,
        'KAPPA': 1.0,
        'THETA': 0.04,
        'ALPHA': 0.01,
        'V0' : 0.03,
        'RHO': -0.8    }
}

In [ ]:
stochvol_pricing_data.models = analytics.vectorConstModel(len(spec['BASKET']['UNDERLYINGS']))
for i, key in enumerate(spec['BASKET']['UNDERLYINGS']):
    stochvol_pricing_data.models[i] = model_converter.stochvolmodel_from_dict(heston[key], refdate)

## Bewertung
### Einfacher Preiscall

In [ ]:
tic = dt.datetime.now()
stochvol_pr = analytics.price(stochvol_pricing_data)
print('runtime: {}'.format(dt.datetime.now() - tic))
#plot the price
stochvol_pr.getPrice()